# KNN Model

### Group Name: The Big One

### Group Members: Nicholas Parker, Matthew King, and Sean Sturtevant

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

#### Read CSV's into pandas dataframes

In [2]:
hockey = pd.read_csv("data/processed/hockey.csv")

#### Split into train and test sets.

In [3]:
y_train = hockey['Salary']
X_train = hockey.drop('Salary', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)

#### Select nonpredictive_feature and separate numerical and categorical variables.

In [4]:
# nonpredictive_features = []
# negImportanceDropList = ['Position', 'Team', 'PEND', 'TOIX', 'GA', 'xGA', 'iSCF', 'iPEND', 'sDist.1', 'iHF.1', 'PDO', 'Hand', 'SCA', 'iTKA.1', 'SA', 'IPP%', 'ixG', 'FA', 'Pace', 'iGVA.1', 'SV%', 'RBF', 'PENT', 'F/60', 'GVA', 'TKA', 'FOW', 'Diff/60']
nonpredictive_features = ['ENG', 'Wide', 'Over', 'PSG', 'PSA', 'S.Dflct', 'G.Bkhd', 'Post', 'G.Dflct', 'CBar ', 'G.Slap', 'G.Snap', 'G.Wrst', 'G.Wrap', 'G.Tip', 'S.Bkhd', 'Min', 'S.Slap', 'Misc', 'Noise', 'DAP', 'Grit', 'PS', 'DPS', 'OPS', 'DSA', 'DSF', 'Game', 'Match', 'S.Snap', 'Maj', '1G', 'NPD', 'iPenDf', 'iPenD', 'iPenT', 'S.Wrst', 'S.Wrap', 'S.Tip', 'GWG', 'FOL.Down', 'OTG', 'PIM', 'iSF.1', 'iCF.1', 'Diff', 'Pct%', 'FOL.Close', 'TOI/GP.1', 'TOI/GP', 'TOI', 'Shifts', 'E+/-', 'sDist', '+/-', 'PTS', 'A2', 'A1', 'A', 'G', 'GP', 'Wt', 'Ht', 'iSF.2', 'Age', 'iFOW', 'iBLK', 'iFOL', 'dzFOL', 'nzFOW', 'nzFOL', 'ozFOW', 'ozFOL', 'dzFOW', 'FOL.Up', 'FOW.Up', 'iTKA', 'iGVA', 'iMiss', 'FOW.Down', 'iHF', 'FOW.Close', 'FO%', 'Position', 'Team', 'PEND', 'TOIX', 'GA', 'xGA', 'iSCF', 'iPEND', 'sDist.1', 'iHF.1', 'PDO', 'Hand', 'SCA', 'iTKA.1', 'SA', 'IPP%', 'ixG', 'FA', 'Pace', 'iGVA.1', 'SV%', 'RBF', 'PENT', 'F/60', 'GVA', 'TKA', 'FOW', 'Diff/60']
# RedundantColDropList = ['TOI/GP', 'iCF', 'iSF', 'iSF.1', 'sDist', 'iHF', 'iGVA', 'iTKA', 'iBLK', 'iFOW', 'iFOL']

numeric = [feature for feature in X_train.columns if np.issubdtype(X_train[feature], np.number) 
                      and feature not in nonpredictive_features]
categorical = [feature for feature in X_train.columns if feature not in numeric
                       and feature not in nonpredictive_features]

In [5]:
def make_pipeline(knn=None):
    """
    Creates pipeline that performs separate transformations on the categorical and numerical features.
    """
    
    numeric_features = numeric
    numeric_transformer = Pipeline(steps=[
        ('imputer', impute.SimpleImputer(strategy='median')),
        ('normalizer', preprocessing.Normalizer())])

#     categorical_features = categorical
#     categorical_transformer = Pipeline(steps=[
#         ('imputer', impute.SimpleImputer(strategy='constant', fill_value='unknown')),
#         ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = compose.ColumnTransformer(transformers=[
        ('numerical', numeric_transformer, numeric_features)])

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('knn', knn)])
    
    return pipeline

knn = KNeighborsRegressor()
pipeline = make_pipeline(knn)

In [6]:
def make_random_cv():
    """
    Define hyperparameter search space
    Instantiate RandomizedSearchCV with the pipeline.
    """
    
#     algo = ['ball_tree', 'kd_tree', 'auto']
    weights = ['distance', 'uniform']
    neighbors = [5, 6, 7, 8, 9, 10, 15]
    hyperparameters = dict(
                          knn__n_neighbors=neighbors,
                          knn__weights=weights)
    
    reg_random_cv = RandomizedSearchCV(pipeline, 
                                       hyperparameters, 
                                       cv=5, 
                                       n_iter=15, 
                                       verbose=1,
                                       random_state=42)
    
    return reg_random_cv

model = make_random_cv()
model.fit(X_train, y_train)

/Users/MatthewKing/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 14 is smaller than n_iter=15. Running 14 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    0.9s finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('preprocessor',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('numerical',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('imputer',
                                                                                                SimpleImputer(add_indicator=False,
                                                               

#### Report and and observe evaluation metrics

In [7]:
def place_commas(number): 
    return ("{:,}".format(number)) 

#### Median Absolute Error

In [8]:
median_absolute_error_scorer = make_scorer(metrics.median_absolute_error)
cross_val_score(model.best_estimator_, 
                X_train, 
                y_train, 
                scoring=median_absolute_error_scorer,
                cv=10)

array([511865.79670705, 406366.55776634, 774824.79832331, 627865.1442681 ,
       677412.72878431, 682604.06552005, 372264.94354526, 750237.22557782,
       437862.94302385, 624692.59372897])

In [9]:
y_pred = model.best_estimator_.predict(X_train)
medae_value_train = place_commas(round(metrics.median_absolute_error(y_train, y_pred), 2))
print(f"${medae_value_train} medae on train dataset")

y_pred = model.best_estimator_.predict(X_test)
medae_value_test = place_commas(round(metrics.median_absolute_error(y_test, y_pred), 2))
print(f"${medae_value_test:} medae on test dataset")

$0.0 medae on train dataset
$639,640.39 medae on test dataset


#### Root Mean Squared Error

In [10]:
y_pred = model.best_estimator_.predict(X_train)
rmse_value_train = place_commas(round(np.sqrt(metrics.mean_squared_error(y_train, y_pred)), 2))
print(f"${rmse_value_train} rmse on train dataset")

y_pred = model.best_estimator_.predict(X_test)
rmse_value_test = place_commas(round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)), 2))
print(f"${rmse_value_test} rmse on test dataset")

$0.0 rmse on train dataset
$1,610,360.39 rmse on test dataset


In [11]:
y_pred = model.best_estimator_.predict(X_train)
medae_value_train = metrics.(y_train, y_pred)
print(f"${medae_value_train:.4f} medae on train dataset")

y_pred = model.best_estimator_.predict(X_test)
medae_value_test = metrics.mean_squared_error(y_test, y_pred)
print(f"${medae_value_test:.4f} medae on test dataset")

SyntaxError: invalid syntax (<ipython-input-11-d4c512683f52>, line 2)